## Spark MLLib - Regressão Linear

In [1]:
# Spark Session - usada quando se trabalha com Dataframes no Spark
spSession = SparkSession.builder.master('local').appName('DSA-SparkMLLib').config('spark.some.config.option', 'session').getOrCreate()

In [2]:
from pyspark.sql import Row
from pyspark.ml.linalg import Vectors
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator

In [3]:
# Carregando os dados e gerando um RDD
carrosRDD = sc.textFile('data/carros.csv')

In [4]:
# Colocando o RDD em cache. Esse processo otimiza a performance.
carrosRDD.cache()

data/carros.csv MapPartitionsRDD[1] at textFile at NativeMethodAccessorImpl.java:0

In [5]:
carrosRDD.count()

399

In [6]:
carrosRDD.take(5)

['MPG,CYLINDERS,DISPLACEMENT,HORSEPOWER,WEIGHT,ACCELERATION,MODELYEAR,NAME',
 '18,8,307,130,3504,12,70,chevrolet chevelle malibu',
 '15,8,350,165,3693,11.5,70,buick skylark 320',
 '18,8,318,150,3436,11,70,plymouth satellite',
 '16,8,304,150,3433,12,70,amc rebel sst']

In [7]:
# Removendo a primeira linha do arquivo (cabeçalho)
carrosRDD = carrosRDD.filter(lambda x: 'DISPLACEMENT' not in x)
carrosRDD.count()

398

## Limpeza dos Dados

In [8]:
# Usando um valor padrão para average HP (que será usado para preencher os valores missing)
mediaHP = sc.broadcast(75.0)

In [9]:
# Função para limpeza dos dados
def limpadados(inputStr):
    global mediaHP
    attList = inputStr.split(',')
    
    # Substitui o caracter ? por um valor
    hpValue = attList[3]
    if hpValue == '?':
        hpValue = mediaHP.value
        
    # Cria uma linha usando a função Row, limpando e convertendo os dados de string para float
    linhas = Row(MPG=float(attList[0]), CYLINDERS=float(attList[1]), DISPLACEMENT=float(attList[2]),
                 HORSEPOWER=float(hpValue), WEIGHT=float(attList[4]), ACCELERATION=float(attList[5]),
                 MODELYEAR=float(attList[6]), NAME=attList[7])
    return linhas

In [10]:
# Executa a função no RDD
carrosRDD3 = carrosRDD.map(limpadados)
carrosRDD3.cache()
carrosRDD3.take(5)

[Row(ACCELERATION=12.0, CYLINDERS=8.0, DISPLACEMENT=307.0, HORSEPOWER=130.0, MODELYEAR=70.0, MPG=18.0, NAME='chevrolet chevelle malibu', WEIGHT=3504.0),
 Row(ACCELERATION=11.5, CYLINDERS=8.0, DISPLACEMENT=350.0, HORSEPOWER=165.0, MODELYEAR=70.0, MPG=15.0, NAME='buick skylark 320', WEIGHT=3693.0),
 Row(ACCELERATION=11.0, CYLINDERS=8.0, DISPLACEMENT=318.0, HORSEPOWER=150.0, MODELYEAR=70.0, MPG=18.0, NAME='plymouth satellite', WEIGHT=3436.0),
 Row(ACCELERATION=12.0, CYLINDERS=8.0, DISPLACEMENT=304.0, HORSEPOWER=150.0, MODELYEAR=70.0, MPG=16.0, NAME='amc rebel sst', WEIGHT=3433.0),
 Row(ACCELERATION=10.5, CYLINDERS=8.0, DISPLACEMENT=302.0, HORSEPOWER=140.0, MODELYEAR=70.0, MPG=17.0, NAME='ford torino', WEIGHT=3449.0)]

## Análise Exploratória dos Dados

In [11]:
# Cria um Dataframe
carrosDF = spSession.createDataFrame(carrosRDD3)

In [15]:
# Estatísticas descritividas
carrosDF.select('MPG', 'CYLINDERS').describe().show()

+-------+-----------------+------------------+
|summary|              MPG|         CYLINDERS|
+-------+-----------------+------------------+
|  count|              398|               398|
|   mean|23.51457286432161| 5.454773869346734|
| stddev|7.815984312565782|1.7010042445332125|
|    min|              9.0|               3.0|
|    max|             46.6|               8.0|
+-------+-----------------+------------------+



In [18]:
# Encontrando a correlação entre a variável target com as variáveis preditoras
for i in carrosDF.columns:
    if not(isinstance(carrosDF.select(i).take(1)[0][0], str)):
        print('Correlação da variável MPG com ', i, carrosDF.stat.corr('MPG', i))

Correlação da variável MPG com  ACCELERATION 0.4202889121016501
Correlação da variável MPG com  CYLINDERS -0.7753962854205548
Correlação da variável MPG com  DISPLACEMENT -0.8042028248058979
Correlação da variável MPG com  HORSEPOWER -0.7747041523498721
Correlação da variável MPG com  MODELYEAR 0.5792671330833091
Correlação da variável MPG com  MPG 1.0
Correlação da variável MPG com  WEIGHT -0.8317409332443347


## Pré-processamento dos dados

In [19]:
# Convertendo para um LabeledPoint (target, Vector(features))
# Remove colunas não relevantes para o modelo ou com baixa correlação
def transformaVar(row):
    obj = (row['MPG'], Vectors.dense([row['ACCELERATION'], row['DISPLACEMENT'], row['WEIGHT']]))
    return obj

In [21]:
# Utiliza o RDD, aplica a função, converte para Dataframe e aplica a função select()
carrosRDD4 = carrosRDD3.map(transformaVar)
carrosDF = spSession.createDataFrame(carrosRDD4, ['label', 'features'])
carrosDF.select('label', 'features').show(10)

+-----+-------------------+
|label|           features|
+-----+-------------------+
| 18.0|[12.0,307.0,3504.0]|
| 15.0|[11.5,350.0,3693.0]|
| 18.0|[11.0,318.0,3436.0]|
| 16.0|[12.0,304.0,3433.0]|
| 17.0|[10.5,302.0,3449.0]|
| 15.0|[10.0,429.0,4341.0]|
| 14.0| [9.0,454.0,4354.0]|
| 14.0| [8.5,440.0,4312.0]|
| 14.0|[10.0,455.0,4425.0]|
| 15.0| [8.5,390.0,3850.0]|
+-----+-------------------+
only showing top 10 rows



In [22]:
carrosRDD4.take(5)

[(18.0, DenseVector([12.0, 307.0, 3504.0])),
 (15.0, DenseVector([11.5, 350.0, 3693.0])),
 (18.0, DenseVector([11.0, 318.0, 3436.0])),
 (16.0, DenseVector([12.0, 304.0, 3433.0])),
 (17.0, DenseVector([10.5, 302.0, 3449.0]))]

## Machine Learning

In [23]:
# Dados de treino e de teste
(dados_treino, dados_teste) = carrosDF.randomSplit([0.7, 0.3])

In [24]:
dados_treino.count()

265

In [25]:
dados_teste.count()

133

In [26]:
# Construindo o modelo com os dados de treino
linearReg = LinearRegression(maxIter=10)
modelo = linearReg.fit(dados_treino)

In [27]:
print(modelo)

LinearRegression_4eba9b09cc4bf7588fff


In [28]:
# Imprimindo as métricas
print('Coeficientes: ' + str(modelo.coefficients))
print('Intercept: ' + str(modelo.intercept))

Coeficientes: [0.2923564326289963,-0.005734439567963864,-0.0067239069723166196]
Intercept: 40.05305795328767


In [29]:
# Previsões com dados de teste
predictions = modelo.transform(dados_teste)
predictions.select('prediction', 'features').show()

+------------------+-------------------+
|        prediction|           features|
+------------------+-------------------+
| 7.658295026952736|[11.5,429.0,4952.0]|
| 8.426249927779246|[12.5,400.0,4906.0]|
| 8.908125788383334|[11.0,440.0,4735.0]|
| 20.52147320003917|[12.0,302.0,3169.0]|
|12.816302896367041|[12.0,350.0,4274.0]|
| 6.706677479942655|[12.0,400.0,5140.0]|
|14.278619142179128|[13.0,350.0,4100.0]|
|12.504497168891895|[13.0,351.0,4363.0]|
|11.721786755622347|[13.5,350.0,4502.0]|
|10.614163923653035|[10.0,455.0,4425.0]|
|  11.7832272440417|[12.0,400.0,4385.0]|
|12.207864635551438|[13.5,318.0,4457.0]|
|10.673846735345304|[13.5,351.0,4657.0]|
|15.382393494779254|[14.5,302.0,4042.0]|
|14.217641049224248|[15.5,304.0,4257.0]|
|16.823051382683357|[10.0,383.0,3563.0]|
| 11.32806753809469|[10.0,429.0,4341.0]|
|  16.5767146309685|[11.5,350.0,3693.0]|
|12.284847204220473|[14.0,350.0,4440.0]|
| 21.68248441751499|[21.0,250.0,3432.0]|
+------------------+-------------------+
only showing top

In [30]:
# Coeficiente de determinação R2
avaliador = RegressionEvaluator(predictionCol = 'prediction', labelCol='label', metricName = 'r2')
avaliador.evaluate(predictions)

0.6789294554138561